# 3 - UNIFICAR DATSETS y EDA

**AUTOR: Fabrizio Ramirez Cutimbo**

**OBJETIVO**

+ Unificar todos los datasests de la Superintendencia de Compañías (directorio, información financiera y ciiu).
+ Aplicar técnicas de anonimización a datos identificadores como EXPEDIENTE y RUC para evitar conflicto de interés.
---

In [97]:
import pandas as pd
import numpy as np

### Carga de datasets

In [98]:
pd_companias = pd.read_csv('Datasets Procesados/directorio_empresas.csv', dtype={'RUC': 'object'})
pd_info_financiera_2022 = pd.read_csv('Datasets Procesados/informacion_financiera_2022.csv', dtype={'RUC': 'object'})
pd_info_financiera_2023 = pd.read_csv('Datasets Procesados/informacion_financiera_2023.csv', dtype={'RUC': 'object'})

### Forma original

In [99]:
print(f"Dimensión Original de directorio de Compañías. \n Filas: {pd_companias.shape[0]} \n Columnas: {pd_companias.shape[1]}")
pd_companias.sample(1)

Dimensión Original de directorio de Compañías. 
 Filas: 127248 
 Columnas: 18


,EXPEDIENTE,RUC,SITUACION_LEGAL,FECHA_CONSTITUCION,TIPO,PAIS,REGION,PROVINCIA,CANTON,CIUDAD,CIIU_NIVEL_1,CIIU_NIVEL_1_DESC,CIIU_NIVEL_3,CIIU_NIVEL_3_DESC,CIIU_NIVEL_4,CIIU_NIVEL_4_DESC,CIIU_NIVEL_6,CIIU_NIVEL_6_DESC
63441,324297,1793152260001,ACTIVA,14/06/2021,SOCIEDAD POR ACCIONES SIMPLIFICADA,ECUADOR,SIERRA,PICHINCHA,QUITO,QUITO,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G479,"VENTA AL POR MENOR NO REALIZADA EN COMERCIOS, ...",G4791,VENTA AL POR MENOR POR CORREO Y POR INTERNET,G4791.00,VENTA AL POR MENOR DE CUALQUIER TIPO DE PRODUC...


In [100]:
print(f"Dimensión Original de datset Información Financiera 2022.\n Filas: {pd_info_financiera_2022.shape[0]} \n Columnas: {pd_info_financiera_2022.shape[1]}" )
pd_info_financiera_2022.sample(1)


Dimensión Original de datset Información Financiera 2022.
 Filas: 119126 
 Columnas: 19


,EXPEDIENTE,RUC,ACTIVO_2022,ACTIVO_CORRIENTE_2022,INVENTARIOS_2022,ACTIVOS_NO_CORRIENTES_2022,PASIVO_2022,PASIVO_CORRIENTE_2022,PASIVO_NO_CORRIENTE_2022,PATRIMONIO_NETO_2022,INGRESOS_ACTIVIDADES_ORDINARIAS_2022,GANANCIA_BRUTA_2022,OTROS_INGRESOS_2022,COSTO_VENTAS_PRODUCCION_2022,GASTOS_2022,UTILIDAD_OPERATIVA_2022,GANACIA_PERDIDA_ANTES_IR_2022,IMPUESTO_RENTA_2022,UTILIDAD_NETA_2022
96054,719489,1792900808001,876.0,876.0,0.0,0.0,0.0,0.0,0.0,876.0,12.0,12.0,0.0,0.0,36.0,-24.0,0.0,0.0,0.0


In [101]:
print(f"Dimensión Original de datset Información Financiera 2023.\n Filas: {pd_info_financiera_2023.shape[0]} \n Columnas: {pd_info_financiera_2023.shape[1]}" )
pd_info_financiera_2023.sample(1)

Dimensión Original de datset Información Financiera 2023.
 Filas: 125515 
 Columnas: 19


,EXPEDIENTE,RUC,ACTIVO_2023,ACTIVO_CORRIENTE_2023,INVENTARIOS_2023,ACTIVOS_NO_CORRIENTES_2023,PASIVO_2023,PASIVO_CORRIENTE_2023,PASIVO_NO_CORRIENTE_2023,PATRIMONIO_NETO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,GANANCIA_BRUTA_2023,OTROS_INGRESOS_2023,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2023,UTILIDAD_OPERATIVA_2023,GANACIA_PERDIDA_ANTES_IR_2023,IMPUESTO_RENTA_2023,UTILIDAD_NETA_2023
12842,76073,0991370935001,335631.92,206351.52,101295.12,129280.4,99927.67,89228.74,10698.93,235704.25,524100.88,301540.28,2700.0,222560.6,302208.66,2031.62,1726.88,379.9,0.0


## Combinación de datasets

In [102]:
print(f"Tamaño inicial de dataset compañias: {pd_companias.shape[0]}")

Tamaño inicial de dataset compañias: 127248


+ Se aplica un LEFT MERGE para conversar todas las compañías y unirlas con la información financiera de 2022 y 2023.

In [103]:
pd_merged = pd.merge(pd_companias, pd_info_financiera_2022, left_on='RUC', right_on='RUC', how='left')
pd_merged = pd.merge(pd_merged, pd_info_financiera_2023, left_on='RUC', right_on='RUC', how='left')

+ Se revisan los NAN en las columnas del año 2022 y 2023

In [104]:
# Se valida respecto a la columna ACTIVO como referencia
pd_validation_2022 = pd_merged[pd_merged['ACTIVO_2022'].isna()]
pd_validation_2023 = pd_merged[pd_merged['ACTIVO_2023'].isna()]
pd_validation_both = pd_merged[pd_merged[['ACTIVO_2022', 'ACTIVO_2023']].isna().all(axis=1)]

print(f'Registros con valonres NaN en Información Financiera 2022: {pd_validation_2022.shape[0]}')
print(f'Registros con valonres NaN en Información Financiera 2023: {pd_validation_2023.shape[0]}')
print(f'Registros con valonres NaN en Información Financiera 2022 y 2023: {pd_validation_both.shape[0]}')

Registros con valonres NaN en Información Financiera 2022: 14849
Registros con valonres NaN en Información Financiera 2023: 6380
Registros con valonres NaN en Información Financiera 2022 y 2023: 213


+ Si no existe un valor en al columna 2022 se reemplaza el correspondiente del año 2023 y viceversa

In [105]:
# Lista base de columnas a revisar
columnas_reemplazo = [
    'ACTIVO', 'ACTIVO_CORRIENTE', 'INVENTARIOS', 'ACTIVOS_NO_CORRIENTES', 
    'PASIVO', 'PASIVO_CORRIENTE', 'PASIVO_NO_CORRIENTE', 
    'PATRIMONIO_NETO',
    'INGRESOS_ACTIVIDADES_ORDINARIAS', 'GANANCIA_BRUTA', 'OTROS_INGRESOS',
    'COSTO_VENTAS_PRODUCCION', 'GASTOS', 
    'UTILIDAD_OPERATIVA', 'GANACIA_PERDIDA_ANTES_IR', 'IMPUESTO_RENTA', 'UTILIDAD_NETA'
]

# Procesar cada columna base
for columna in columnas_reemplazo:
    col_anio1 = f"{columna}_2022"
    col_anio2 = f"{columna}_2023"
    
    # Verificar que ambas columnas existan
    if col_anio1 in pd_merged.columns and col_anio2 in pd_merged.columns:

        # Rellenar NaN del año1 con los valores del año2
        pd_merged[col_anio1] = pd_merged[col_anio1].fillna(pd_merged[col_anio2])
        # Rellenar NaN del año2 con los valores del año1
        pd_merged[col_anio2] = pd_merged[col_anio2].fillna(pd_merged[col_anio1])

In [106]:
pd_validation_2022 = pd_merged[pd_merged['ACTIVO_2022'].isna()]
pd_validation_2023 = pd_merged[pd_merged['ACTIVO_2023'].isna()]
pd_validation_both = pd_merged[pd_merged[['ACTIVO_2022', 'ACTIVO_2023']].isna().all(axis=1)]

print(f'Registros con valonres NaN en Información Financiera 2022: {pd_validation_2022.shape[0]}')
print(f'Registros con valonres NaN en Información Financiera 2023: {pd_validation_2023.shape[0]}')
print(f'Registros con valonres NaN en Información Financiera 2022 y 2023: {pd_validation_both.shape[0]}')

Registros con valonres NaN en Información Financiera 2022: 213
Registros con valonres NaN en Información Financiera 2023: 213
Registros con valonres NaN en Información Financiera 2022 y 2023: 213


+ Si no existe un valor en la columna del 2022 y 2023 se elimina la fila

In [107]:
# Filtrar filas donde las columnas de 2022 y 2023 tengan datos
pd_merged_cleaned = pd_merged.dropna(subset=['ACTIVO_2022', 'ACTIVO_2023'], how='all')
print(f"Tamaño luego de combinación y limpieza: {pd_merged_cleaned.shape[0]}")


Tamaño luego de combinación y limpieza: 127035


- Estadisticas del dataset antes de filtros

In [108]:
# función para imprimir Describe() con dos decimales
def describe_formateado(dataset, columns):

    # Selección de las columnas relevantes
    description = dataset[columns].describe()
    # Aplicar formato a los valores en el DataFrame de descripción
    formatted_description = description.applymap(lambda x: f"{x:.3f}")
    return formatted_description

describe_formateado(pd_merged_cleaned, ['ACTIVO_2023', 'ACTIVO_CORRIENTE_2023', 'INVENTARIOS_2023', 'ACTIVOS_NO_CORRIENTES_2023', 
                                        'PASIVO_2023', 'PASIVO_CORRIENTE_2023', 'PASIVO_NO_CORRIENTE_2023', 'PATRIMONIO_NETO_2023', 
                                        'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'GANANCIA_BRUTA_2023', 'OTROS_INGRESOS_2023', 
                                        'COSTO_VENTAS_PRODUCCION_2023', 'GASTOS_2023', 
                                        'UTILIDAD_OPERATIVA_2023', 'GANACIA_PERDIDA_ANTES_IR_2023', 'IMPUESTO_RENTA_2023', 'UTILIDAD_NETA_2023'])

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,ACTIVO_2023,ACTIVO_CORRIENTE_2023,INVENTARIOS_2023,ACTIVOS_NO_CORRIENTES_2023,PASIVO_2023,PASIVO_CORRIENTE_2023,PASIVO_NO_CORRIENTE_2023,PATRIMONIO_NETO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,GANANCIA_BRUTA_2023,OTROS_INGRESOS_2023,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2023,UTILIDAD_OPERATIVA_2023,GANACIA_PERDIDA_ANTES_IR_2023,IMPUESTO_RENTA_2023,UTILIDAD_NETA_2023
count,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000,127035.000
mean,1315181.990,608494.798,169564.999,703721.890,727844.189,459841.125,267315.910,587294.253,1196633.256,400547.111,17025.065,792815.581,333732.064,-1017375430.669,1472451257.664,205627.647,606206.230
std,18065052.277,6951427.961,2431568.465,12778039.128,8727914.602,5540254.113,4219749.491,11240806.547,17122838.915,6362370.186,318395.591,12866093.954,4328480.239,362642977885.077,750265467735.534,67090557.514,201273728.659
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-32075310.250,0.000,-45545741.750,0.000,0.000,0.000,-129253100000002.000,-129253100000002.000,0.000,-66586053.000
25%,1043.520,897.340,0.000,0.000,0.000,0.000,0.000,800.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,26459.920,12204.180,0.000,337.070,8824.170,4818.300,0.000,5000.000,13813.370,5536.980,0.000,0.000,8577.840,0.000,0.000,0.000,0.000
75%,223331.710,112301.315,0.000,45310.885,121604.710,74487.145,2220.910,58653.875,197464.605,103274.345,0.000,21224.500,97047.630,6866.340,2128.425,389.895,15.930
max,2564376140.550,689175916.390,292689998.020,1994256308.050,778408645.990,483177041.180,550443293.360,1785967494.560,2483015099.250,955033665.520,42102526.230,1817217513.450,700779280.410,527655725.850,207032575000039.000,23911794125.000,71735382375.000


**IMPORTANTE** Se puede observar que varias variables como los "INVENTARIOS", "IMPUESTO_RENTA_2023".
Recien en el tercer Cuartil 3 tiene un valor diferente a cero, esto ocurre porque la gran mayoría de valores tienen el valor 0

## Limpieza casos anómalos
+ Se eliminan empresas que no hayan registrado ningun tipo de ingreso en 2023  y 2022.
+ Los **INGRESOS** son un dato fundamental pues se utilizarán en el Location Quotient.

In [109]:
condicion = (
    (pd_merged_cleaned['INGRESOS_ACTIVIDADES_ORDINARIAS_2022'] == 0) &
    (pd_merged_cleaned['GANANCIA_BRUTA_2022'] == 0) &
    (pd_merged_cleaned['OTROS_INGRESOS_2022'] == 0) &
    (pd_merged_cleaned['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] == 0) &
    (pd_merged_cleaned['GANANCIA_BRUTA_2023'] == 0) &
    (pd_merged_cleaned['OTROS_INGRESOS_2023'] == 0)
)

# Conteo registros que cumplen con la condición
pd_empresas_sin_ingresos = pd_merged_cleaned[condicion]
registros_cumplen_condicion = pd_merged_cleaned[condicion].shape[0]
print(f"Cantidad de Empresas ACTIVAS que NO registran INGRESOS: {registros_cumplen_condicion}")

Cantidad de Empresas ACTIVAS que NO registran INGRESOS: 38305


In [110]:
print(f"% De Empresas que no registran ingresos: ")
round(registros_cumplen_condicion/pd_merged_cleaned.shape[0]*100, 2)

% De Empresas que no registran ingresos: 


30.15

In [111]:
print(pd_empresas_sin_ingresos.shape[0])
pd_empresas_sin_ingresos.sample(1)

38305


,EXPEDIENTE_x,RUC,SITUACION_LEGAL,FECHA_CONSTITUCION,TIPO,PAIS,REGION,PROVINCIA,CANTON,CIUDAD,...,PATRIMONIO_NETO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,GANANCIA_BRUTA_2023,OTROS_INGRESOS_2023,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2023,UTILIDAD_OPERATIVA_2023,GANACIA_PERDIDA_ANTES_IR_2023,IMPUESTO_RENTA_2023,UTILIDAD_NETA_2023
55639,313913,1792985617001,ACTIVA,14/05/2019,ANÓNIMA,ECUADOR,SIERRA,PICHINCHA,QUITO,QUITO,...,5166.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
describe_formateado(pd_empresas_sin_ingresos, ['ACTIVO_2022', 'ACTIVO_2023', 'PASIVO_2022', 'PASIVO_2023', 
                                               'INGRESOS_ACTIVIDADES_ORDINARIAS_2022', 'INGRESOS_ACTIVIDADES_ORDINARIAS_2023',                                               
                                               'COSTO_VENTAS_PRODUCCION_2022', 'COSTO_VENTAS_PRODUCCION_2023', 'GASTOS_2022', 'GASTOS_2023',
                                               'UTILIDAD_OPERATIVA_2022', 'UTILIDAD_OPERATIVA_2023'])

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,ACTIVO_2022,ACTIVO_2023,PASIVO_2022,PASIVO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2022,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,COSTO_VENTAS_PRODUCCION_2022,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2022,GASTOS_2023,UTILIDAD_OPERATIVA_2022,UTILIDAD_OPERATIVA_2023
count,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000,38305.000
mean,110753.309,120728.150,49900.450,56829.307,0.000,0.000,1129.861,72.028,2157.339,1524.639,-1729.098,-1462.588
std,2418186.448,2486619.909,788761.939,901070.178,0.000,0.000,166335.135,6520.317,122243.307,35454.828,119799.998,35507.384
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-22744107.790,-5269720.630
25%,400.000,400.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,800.000,800.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,5452.050,5498.580,100.000,144.390,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,296203522.310,296203522.310,81528121.940,107568900.000,0.000,0.000,32282668.930,1172145.650,22744107.790,5269720.630,586964.750,586964.750


**Importante** Se observan empresas que durante dos años consecutivos no han generado ningun tipo de ingreso por actividades ordinarias ni extraordinarias, la gran mayoría tampoco tiene costos ni gastos. Potencial revisión para trabajos futuros.

+ Eliminar las filas que cumplen con la condición

**IMPORTANTE**, aquí se observa que se pierde un conjunto considerable de Empresas

In [113]:
print(f'Tamaño antes del filtro: {pd_merged_cleaned.shape[0]}')

# Eliminar las filas que cumplen con la condición
pd_merged_cleaned = pd_merged_cleaned[~condicion]

print(f"Tamaño después de filtro: {pd_merged_cleaned.shape[0]}")

Tamaño antes del filtro: 127035
Tamaño después de filtro: 88730


+ Eliminar empresas cuyo impuesto a la renta sea mayor o igual que los Ingresos (lo cual es extraño pues el IR es una fracción de los ingresos)

In [114]:
condicion = (
    (pd_merged_cleaned['IMPUESTO_RENTA_2023'] > (1.0 * (
        pd_merged_cleaned['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] +
        pd_merged_cleaned['GANANCIA_BRUTA_2023'] +
        pd_merged_cleaned['OTROS_INGRESOS_2023']
    ))) &
    (pd_merged_cleaned['UTILIDAD_OPERATIVA_2023'] > 0)
)

# Conteo registros que cumplen con la condición
registros_cumplen_condicion = pd_merged_cleaned[condicion].shape[0]
print(f"Cantidad de registros que cumplen la condición: {registros_cumplen_condicion}")

Cantidad de registros que cumplen la condición: 57


In [115]:
print(f'Tamaño antes del filtro: {pd_merged_cleaned.shape[0]}')

# Eliminar las filas que cumplen con la condición
pd_merged_cleaned = pd_merged_cleaned[~condicion]

print(f"Tamaño después de filtro: {pd_merged_cleaned.shape[0]}")

Tamaño antes del filtro: 88730
Tamaño después de filtro: 88673


### Seleccionar y Ordenar Columnas

In [116]:
pd_merged_cleaned.columns

Index(['EXPEDIENTE_x', 'RUC', 'SITUACION_LEGAL', 'FECHA_CONSTITUCION', 'TIPO',
       'PAIS', 'REGION', 'PROVINCIA', 'CANTON', 'CIUDAD', 'CIIU_NIVEL_1',
       'CIIU_NIVEL_1_DESC', 'CIIU_NIVEL_3', 'CIIU_NIVEL_3_DESC',
       'CIIU_NIVEL_4', 'CIIU_NIVEL_4_DESC', 'CIIU_NIVEL_6',
       'CIIU_NIVEL_6_DESC', 'EXPEDIENTE_y', 'ACTIVO_2022',
       'ACTIVO_CORRIENTE_2022', 'INVENTARIOS_2022',
       'ACTIVOS_NO_CORRIENTES_2022', 'PASIVO_2022', 'PASIVO_CORRIENTE_2022',
       'PASIVO_NO_CORRIENTE_2022', 'PATRIMONIO_NETO_2022',
       'INGRESOS_ACTIVIDADES_ORDINARIAS_2022', 'GANANCIA_BRUTA_2022',
       'OTROS_INGRESOS_2022', 'COSTO_VENTAS_PRODUCCION_2022', 'GASTOS_2022',
       'UTILIDAD_OPERATIVA_2022', 'GANACIA_PERDIDA_ANTES_IR_2022',
       'IMPUESTO_RENTA_2022', 'UTILIDAD_NETA_2022', 'EXPEDIENTE',
       'ACTIVO_2023', 'ACTIVO_CORRIENTE_2023', 'INVENTARIOS_2023',
       'ACTIVOS_NO_CORRIENTES_2023', 'PASIVO_2023', 'PASIVO_CORRIENTE_2023',
       'PASIVO_NO_CORRIENTE_2023', 'PATRIMONIO_NETO

In [117]:
pd_dataset_final = pd_merged_cleaned[['EXPEDIENTE_x', 'RUC', 'SITUACION_LEGAL', 'FECHA_CONSTITUCION', 'TIPO', 'PAIS', 'REGION', 'PROVINCIA', 'CANTON', 'CIUDAD', 
       'CIIU_NIVEL_1', 'CIIU_NIVEL_1_DESC', 'CIIU_NIVEL_3', 'CIIU_NIVEL_3_DESC', 'CIIU_NIVEL_4', 'CIIU_NIVEL_4_DESC', 'CIIU_NIVEL_6', 'CIIU_NIVEL_6_DESC',
       # Información Financiera 2022
       'ACTIVO_2022', 'ACTIVO_CORRIENTE_2022', 'INVENTARIOS_2022', 'ACTIVOS_NO_CORRIENTES_2022', 
       'PASIVO_2022', 'PASIVO_CORRIENTE_2022', 'PASIVO_NO_CORRIENTE_2022', 'PATRIMONIO_NETO_2022',
       'INGRESOS_ACTIVIDADES_ORDINARIAS_2022', 'GANANCIA_BRUTA_2022', 'OTROS_INGRESOS_2022', 
       'COSTO_VENTAS_PRODUCCION_2022', 'GASTOS_2022', 
       'UTILIDAD_OPERATIVA_2022', 'GANACIA_PERDIDA_ANTES_IR_2022', 'IMPUESTO_RENTA_2022', 'UTILIDAD_NETA_2022', 
       # Información Financiera 2023
       'ACTIVO_2023', 'ACTIVO_CORRIENTE_2023','INVENTARIOS_2023', 'ACTIVOS_NO_CORRIENTES_2023', 
       'PASIVO_2023', 'PASIVO_CORRIENTE_2023', 'PASIVO_NO_CORRIENTE_2023', 'PATRIMONIO_NETO_2023', 
       'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'GANANCIA_BRUTA_2023', 'OTROS_INGRESOS_2023', 
       'COSTO_VENTAS_PRODUCCION_2023', 'GASTOS_2023', 
       'UTILIDAD_OPERATIVA_2023', 'GANACIA_PERDIDA_ANTES_IR_2023', 'IMPUESTO_RENTA_2023', 'UTILIDAD_NETA_2023']]

In [118]:
# Renombrar columna
pd_dataset_final = pd_dataset_final.rename(columns={
    'EXPEDIENTE_x':'EXPEDIENTE',
    'NOMBRE_x':'NOMBRE'
})
print(f"Tamaño del datset: {pd_dataset_final.shape[0]}")

Tamaño del datset: 88673


## Ajustar valores pequeños

- Se revisan las columnas que se utilizarán para calcular variaciones e indicadores financieros.
- Dado que las variaciones e indicadores financieros realizan **divisiones**, además de controlar la división para 0, también se debe validar que no existan valores muy pequeños que puedan incrementar considerablemente el numerador.
- Si se detectan valores entre -1 y 1, excluyendo el 0, este valor se reemplazará redondeará.

In [119]:
# Columnas a Revisar
columns_revision = ['ACTIVO_2022', 'ACTIVO_CORRIENTE_2022', 'INVENTARIOS_2022', 'ACTIVOS_NO_CORRIENTES_2022', 
                    'PASIVO_2022', 'PASIVO_CORRIENTE_2022', 'PASIVO_NO_CORRIENTE_2022', 'PATRIMONIO_NETO_2022',
                    'INGRESOS_ACTIVIDADES_ORDINARIAS_2022', 'GANANCIA_BRUTA_2022', 'OTROS_INGRESOS_2022', 
                    'COSTO_VENTAS_PRODUCCION_2022', 'GASTOS_2022', 
                    'UTILIDAD_OPERATIVA_2022', 'GANACIA_PERDIDA_ANTES_IR_2022', 'IMPUESTO_RENTA_2022', 'UTILIDAD_NETA_2022', 
                    'ACTIVO_2023', 'ACTIVO_CORRIENTE_2023','INVENTARIOS_2023', 'ACTIVOS_NO_CORRIENTES_2023', 
                    'PASIVO_2023', 'PASIVO_CORRIENTE_2023', 'PASIVO_NO_CORRIENTE_2023', 'PATRIMONIO_NETO_2023', 
                    'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'GANANCIA_BRUTA_2023', 'OTROS_INGRESOS_2023', 
                    'COSTO_VENTAS_PRODUCCION_2023', 'GASTOS_2023', 
                    'UTILIDAD_OPERATIVA_2023', 'GANACIA_PERDIDA_ANTES_IR_2023', 'IMPUESTO_RENTA_2023', 'UTILIDAD_NETA_2023']

# Crear una máscara booleana para valores entre -1 y 1, excluyendo el 0
pd_dataset_anomalos = pd_dataset_final[columns_revision].apply(
    lambda col: col.map(lambda x: -1 < x < 1 and x != 0)
)

# Filtrar las filas donde al menos una columna cumpla la condición
pd_dataset_anomalos = pd_dataset_final[pd_dataset_anomalos.any(axis=1)]

# Mostrar los registros encontrados
print(f"Registros anómalos con valores entre -1 y -1 (Excluyendo 0): {pd_dataset_anomalos.shape[0]}")

Registros anómalos con valores entre -1 y -1 (Excluyendo 0): 2741


In [120]:
pd_dataset_anomalos['OTROS_INGRESOS_2023'].head(3)

52        0.02
166       0.29
255    8548.00
Name: OTROS_INGRESOS_2023, dtype: float64

+ Redondear números anómalos entre -1 y 1

In [121]:
for col in columns_revision:
    # Aplicamos el redondeo solo a los valores entre -1 y 1, excluyendo el 0
    pd_dataset_final[col] = pd_dataset_final[col].apply(lambda x: round(x) if -1 < x < 1 and x != 0 else x)


In [122]:
# Mostrar el dataset actualizado
pd_dataset_final[columns_revision].head(3)

,ACTIVO_2022,ACTIVO_CORRIENTE_2022,INVENTARIOS_2022,ACTIVOS_NO_CORRIENTES_2022,PASIVO_2022,PASIVO_CORRIENTE_2022,PASIVO_NO_CORRIENTE_2022,PATRIMONIO_NETO_2022,INGRESOS_ACTIVIDADES_ORDINARIAS_2022,GANANCIA_BRUTA_2022,...,PATRIMONIO_NETO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,GANANCIA_BRUTA_2023,OTROS_INGRESOS_2023,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2023,UTILIDAD_OPERATIVA_2023,GANACIA_PERDIDA_ANTES_IR_2023,IMPUESTO_RENTA_2023,UTILIDAD_NETA_2023
0,1.360052e+06,7.163018e+04,11258.3,1.288422e+06,5.385453e+04,9.009440e+03,4.484509e+04,1.306197e+06,1.735839e+05,0.00,...,1.266879e+06,2.194414e+05,46641.17,4.64,1.728002e+05,84081.25,-37435.44,-39611.39,0.00,-39611.39
1,4.590456e+08,2.417680e+08,131698005.6,2.172776e+08,2.171298e+08,1.099055e+08,1.072242e+08,2.419159e+08,3.603875e+08,86220933.34,...,2.548553e+08,3.064160e+08,73315167.19,1889612.83,2.331008e+08,47084162.46,28120617.56,25146686.50,5283415.16,20168262.71
2,1.122447e+07,1.089344e+07,6936254.4,3.310313e+05,8.492088e+06,1.861889e+06,6.630199e+06,2.732383e+06,1.223285e+07,0.00,...,3.143337e+06,1.317747e+07,4239130.41,373696.26,8.938343e+06,4615514.26,-2687.59,-2687.59,114307.88,-116995.47


In [123]:
describe_formateado(pd_dataset_final, columns_revision)

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,ACTIVO_2022,ACTIVO_CORRIENTE_2022,INVENTARIOS_2022,ACTIVOS_NO_CORRIENTES_2022,PASIVO_2022,PASIVO_CORRIENTE_2022,PASIVO_NO_CORRIENTE_2022,PATRIMONIO_NETO_2022,INGRESOS_ACTIVIDADES_ORDINARIAS_2022,GANANCIA_BRUTA_2022,...,PATRIMONIO_NETO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,GANANCIA_BRUTA_2023,OTROS_INGRESOS_2023,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2023,UTILIDAD_OPERATIVA_2023,GANACIA_PERDIDA_ANTES_IR_2023,IMPUESTO_RENTA_2023,UTILIDAD_NETA_2023
count,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,...,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000
mean,1726009.521,808556.672,231847.656,8589531556.590,965853.223,599722.571,349011.327,760247.159,1603708.744,418841.288,...,813756.707,1714311.180,573829.131,24390.502,1135765.448,477451.002,-1457515048.820,2109457441.695,24920.530,59740.959
std,20341261.292,8161705.123,3047998.370,1808447030904.972,10162045.125,6445270.073,4863756.837,12219708.159,19792444.332,8467804.811,...,13380082.220,20473081.730,7608744.898,380859.690,15387090.478,5174205.837,434055503870.740,898010325864.988,577826.948,2043023.681
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-50598102.920,0.000,-31931425.720,...,-32075310.250,0.000,-45545741.750,0.000,0.000,0.000,-129253100000002.000,-129253100000002.000,0.000,-66586053.000
25%,7844.390,3188.230,0.000,0.000,1351.930,370.930,0.000,1000.000,2875.000,0.000,...,1350.000,9200.000,3027.720,0.000,0.000,5200.000,0.000,0.000,0.000,0.000
50%,62620.140,30733.660,0.000,2600.850,28944.010,15519.740,0.000,13207.000,52727.250,0.000,...,17216.900,80000.000,42444.980,0.000,0.000,41025.350,1621.120,21.000,10.970,0.000
75%,354426.520,184015.000,3949.260,75539.470,210814.560,123940.150,17899.370,98668.470,317060.290,20073.400,...,113430.430,404287.460,203846.560,0.000,104913.020,189279.470,17848.600,8493.950,1782.930,1890.530
max,2480403867.050,701635273.030,319412262.000,382419999999998.000,939074159.340,531439730.650,557873638.640,1680022314.600,2355580171.130,1162299146.300,...,1785967494.560,2483015099.250,955033665.520,42102526.230,1817217513.450,700779280.410,527655725.850,207032575000039.000,93414106.460,353861446.410


In [124]:
print(f"Tamaño del datset luego de Filtros: {pd_dataset_final.shape[0]}")

Tamaño del datset luego de Filtros: 88673


## Enriquecimiento: Crear nuevas columnas

### Variación de Cuentas Financieras

+ **Variación de Cuentas Financieras** respecto al año anterior
+ Considerar que hay empresas que tienen cuentas en 0, incluso Activos. Para prevenir divisiones para 0, si el divisor es 0, entonces se establece automáticamente "0", que en teoría indica no hay variación: (10 - 10) / 10 = 0

In [125]:
pd_dataset_final['VAR_ACTIVO'] = pd_dataset_final.apply(lambda row: round(((row['ACTIVO_2023'] - row['ACTIVO_2022']) / row['ACTIVO_2022']) if row['ACTIVO_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_INVENTARIOS'] = pd_dataset_final.apply(lambda row: round(((row['INVENTARIOS_2023'] - row['INVENTARIOS_2022']) / row['INVENTARIOS_2022']) if row['INVENTARIOS_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_PASIVO'] = pd_dataset_final.apply(lambda row: round(((row['PASIVO_2023'] - row['PASIVO_2022']) / row['PASIVO_2022']) if row['PASIVO_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_INGRESOS'] = pd_dataset_final.apply(lambda row: round(((row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] - row['INGRESOS_ACTIVIDADES_ORDINARIAS_2022']) / row['INGRESOS_ACTIVIDADES_ORDINARIAS_2022']) if row['INGRESOS_ACTIVIDADES_ORDINARIAS_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_COSTO_VENTAS_PRODUCCION'] = pd_dataset_final.apply(lambda row: round(((row['COSTO_VENTAS_PRODUCCION_2023'] - row['COSTO_VENTAS_PRODUCCION_2022']) / row['COSTO_VENTAS_PRODUCCION_2022']) if row['COSTO_VENTAS_PRODUCCION_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_GASTOS'] = pd_dataset_final.apply(lambda row: round(((row['GASTOS_2023'] - row['GASTOS_2022']) / row['GASTOS_2022']) if row['GASTOS_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_UTILIDAD_OPERATIVA'] = pd_dataset_final.apply(lambda row: round(((row['UTILIDAD_OPERATIVA_2023'] - row['UTILIDAD_OPERATIVA_2022']) / row['UTILIDAD_OPERATIVA_2022']) if row['UTILIDAD_OPERATIVA_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_IMPUESTO_RENTA'] = pd_dataset_final.apply(lambda row: round(((row['IMPUESTO_RENTA_2023'] - row['IMPUESTO_RENTA_2022']) / row['IMPUESTO_RENTA_2022']) if row['IMPUESTO_RENTA_2022'] != 0 else 0, 3), axis=1)
pd_dataset_final['VAR_UTILIDAD_NETA'] = pd_dataset_final.apply(lambda row: round(((row['UTILIDAD_NETA_2023'] - row['UTILIDAD_NETA_2022']) / row['UTILIDAD_NETA_2022']) if row['UTILIDAD_NETA_2022'] != 0 else 0, 3), axis=1)


In [126]:
# Visualizar una muestra
pd_dataset_final[['RUC','ACTIVO_2022','ACTIVO_2023','VAR_ACTIVO']].sample(3)

,RUC,ACTIVO_2022,ACTIVO_2023,VAR_ACTIVO
20690,0992351780001,774140.35,774140.35,0.000
10608,1792280796001,287005.46,287794.01,0.003
65869,2390622697001,31611.00,31611.00,0.000


+ Visualizar Estadísticas de la variaciones

**NOTA** Se observan valores altos de desviación estándar

In [127]:
columnas_variacion = ['VAR_ACTIVO', 'VAR_INVENTARIOS', 'VAR_PASIVO', 'VAR_INGRESOS','VAR_COSTO_VENTAS_PRODUCCION', 'VAR_GASTOS', 'VAR_UTILIDAD_OPERATIVA', 'VAR_IMPUESTO_RENTA', 'VAR_UTILIDAD_NETA']
describe_formateado(pd_dataset_final, columnas_variacion)

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,VAR_ACTIVO,VAR_INVENTARIOS,VAR_PASIVO,VAR_INGRESOS,VAR_COSTO_VENTAS_PRODUCCION,VAR_GASTOS,VAR_UTILIDAD_OPERATIVA,VAR_IMPUESTO_RENTA,VAR_UTILIDAD_NETA
count,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000
mean,101.463,1.118,73.563,266.791,11.382,35.910,-37.812,2.034,-41.922
std,13271.725,112.618,7820.046,17142.025,1275.659,1808.019,11285.668,58.843,11255.392
min,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-3332320.110,-1.000,-3332320.110
25%,-0.064,0.000,-0.141,-0.174,0.000,-0.088,-0.694,0.000,-0.181
50%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,0.320,0.000,0.234,0.241,0.000,0.275,0.007,0.000,0.000
max,3261372.120,25282.516,1830396.049,2876418.980,350089.070,358358.124,142010.548,7544.532,142176.608


+ Importante notar que 'VAR_INVENTARIOS', 'VAR_COSTO_VENTAS_PRODUCCION', 'VAR_IMPUESTO_RENTA' en los Q1, Q2 y Q3 tiene valor en 0, lo cual indica que la gran mayoria de valores son "0"

In [128]:
print("Tamaño Dataset")
print(pd_dataset_final.shape)

Tamaño Dataset
(88673, 61)


Aplicar **Rango Inter Cuartílico** (Extendido hasta *10 el Q1 y Q3)

Se utiliza 10 para ser más flexibles y solo excluir valores extremos

In [129]:
# Función para control de Outliers, no se eliminan sino se reemplazan por los limites definidos
def eliminar_outliers_tecnica_iqr(dataset, columns, iqr_multiplier=1.5, lower_quantile=0.25, upper_quantile=0.75):

    cleaned_df = dataset.copy()  # Crear una copia del DataFrame original

    for col in columns:
        # Calcular Q1, Q3 e IQR
        Q1 = cleaned_df[col].quantile(lower_quantile) # Por defecto 25%
        Q3 = cleaned_df[col].quantile(upper_quantile) # Por defecto 75%
        IQR = Q3 - Q1

        # Definir límites inferior y superior
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR

        # Reemplazar valores menores que el límite inferior
        cleaned_df.loc[cleaned_df[col] < lower_bound, col] = lower_bound

        # Reemplazar valores mayores que el límite superior
        cleaned_df.loc[cleaned_df[col] > upper_bound, col] = upper_bound

    return cleaned_df

In [130]:
columns_to_clean = ['VAR_ACTIVO', 'VAR_PASIVO', 
                    'VAR_INGRESOS', 'VAR_GASTOS', 
                    'VAR_UTILIDAD_OPERATIVA', 'VAR_UTILIDAD_NETA']

columns_to_clean_2 = ['VAR_INVENTARIOS', 'VAR_COSTO_VENTAS_PRODUCCION', 'VAR_IMPUESTO_RENTA']

In [131]:
pd_dataset_final = eliminar_outliers_tecnica_iqr(pd_dataset_final, columns_to_clean, 10)

# En Las columnas 'VAR_INVENTARIOS', 'VAR_IMPUESTO_RENTA' se realiza un proceso más flexible del 10% y 90%
pd_dataset_final = eliminar_outliers_tecnica_iqr(pd_dataset_final, columns_to_clean_2, 10, 0.1, 0.9)

In [132]:
columnas = ['VAR_ACTIVO', 'VAR_INVENTARIOS', 'VAR_PASIVO', 'VAR_INGRESOS','VAR_COSTO_VENTAS_PRODUCCION', 'VAR_GASTOS', 'VAR_UTILIDAD_OPERATIVA', 'VAR_IMPUESTO_RENTA', 'VAR_UTILIDAD_NETA']
describe_formateado(pd_dataset_final, columnas)

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,VAR_ACTIVO,VAR_INVENTARIOS,VAR_PASIVO,VAR_INGRESOS,VAR_COSTO_VENTAS_PRODUCCION,VAR_GASTOS,VAR_UTILIDAD_OPERATIVA,VAR_IMPUESTO_RENTA,VAR_UTILIDAD_NETA
count,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000
mean,0.440,0.013,0.315,0.314,0.182,0.356,-0.010,0.338,-0.137
std,1.223,0.389,1.162,1.257,1.396,1.196,2.431,2.313,0.718
min,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-7.704,-1.000,-1.991
25%,-0.064,0.000,-0.141,-0.174,0.000,-0.088,-0.694,0.000,-0.181
50%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,0.320,0.000,0.234,0.241,0.000,0.275,0.007,0.000,0.000
max,4.160,1.876,3.984,4.391,9.565,3.905,7.017,16.567,1.810


+ Importante Notar como se reduce la Desviación Estándar

### Segmento de la Compañía

Se crea una nueva columna para segmentar las empresas de acuerdo con la ley ecuatoriana de segmentación, que es una buena referencia para determinar la dimensión de una empresa:
+ MIPE
+ PYME
+ EMPRESARIAL
+ CORPORARTIVO

In [133]:
def segmento_empresarial(ingresos):
    if ingresos < 100000:
        return 'MICRO'
    elif 100000 <= ingresos < 1000000:
        return 'PEQUENA'
    elif 1000000 <= ingresos < 5000000:
        return 'MEDIANA'
    elif ingresos >= 5000000:
        return 'GRANDE'
    else:
        return 'OTRO'

# Se aplica segmentación en función de los ingresos de Actividades Ordinarias
pd_dataset_final['SEGMENTO'] = pd_dataset_final['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'].apply(segmento_empresarial)


In [134]:
pd_dataset_final[['INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'SEGMENTO']].sample(4)

,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,SEGMENTO
45594,195456.15,PEQUENA
99145,224436.14,PEQUENA
56898,957650.94,PEQUENA
78504,24953.33,MICRO


- Resumen de empresas agrupadas por segmento

In [135]:
pd_dataset_final['SEGMENTO'].value_counts().sum

<bound method Series.sum of SEGMENTO
MICRO      47328
PEQUENA    28556
MEDIANA     8996
GRANDE      3793
Name: count, dtype: int64>

### Indicadores Financieros

| RAZÓN         | INDICADORES                  | FÓRMULA                                           |
|---------------|------------------------------|--------------------------------------------------|
| **Liquidez**  | Liquidez corriente           | Activo Corriente / Pasivo Corriente             |
|               | Prueba ácida                 | (Activo Corriente – Inventarios) / Pasivo Corriente |
| **Solvencia** | Endeudamiento del activo     | Pasivo Total / Activo Total                     |
|               | Endeudamiento patrimonial    | Pasivo Total / Patrimonio                       |
|               | Endeudamiento del Activo fijo| Patrimonio / Activo Fijo                        |
|               | Endeudamiento a corto plazo  | Pasivo Corriente / Pasivo Total                 |
|               | Endeudamiento a largo plazo  | Pasivo No Corriente / Pasivo Total              |
| **Gestión**   | Rotación de ventas           | Ventas / Activo Total                           |
| **Rentabilidad** | Margen Bruto              | (Ventas - Costo De Ventas) / Ventas             |
|               | Margen Operacional           | Utilidad Operacional / Ventas                   |
|               | ROE                          | Utilidad Neta / Patrimonio                      |
|               | ROA                          | Utilidad Neta / Activo Total                    |

* Utilidad Neta: Después del 15% de trabajadores e impuesto a la renta (Equivale a la "Cuenta 607")


**Nota** Si en el cálculo de algún Indicador Financiero resulta en división para 0. Entonces se establecerá el valor 1, lo cual en la mayoría de casos indica un valor neutro en la mayoría de interpretaciones.

Este tipo de formulas, son susceptibles a outliers, pues si una empresa registra 1 dolar de activos, patrimonio, pasivo, etc. Las divisiones generarán número muy grandes fuera de los rangos interpretativos normales. Será necesario aplicar técnicas de control de outliers luego del cálculo

Como trabajo futuro se podría, establecer estos casos como NaN y especificar el valor promedio del sector

In [136]:
# LIQUIDEZ
pd_dataset_final['IF_PRUEBA_ACIDA'] = pd_dataset_final.apply(lambda row: round(((row['ACTIVO_CORRIENTE_2023'] - row['INVENTARIOS_2023']) / row['PASIVO_CORRIENTE_2023']) if row['PASIVO_CORRIENTE_2023'] != 0 else 1, 3), axis=1)

#  SOLVENCIA
pd_dataset_final['IF_ENDEUDAMIENTO_ACTIVO'] = pd_dataset_final.apply(lambda row: round((row['PASIVO_2023'] / row['ACTIVO_2023']) if row['ACTIVO_2023'] != 0 else 1, 3), axis=1)
pd_dataset_final['IF_APALANCAMIENTO'] = pd_dataset_final.apply(lambda row: round((row['ACTIVO_2023'] / row['PATRIMONIO_NETO_2023']) if row['PATRIMONIO_NETO_2023'] != 0 else 1, 3), axis=1)

# GESTION
pd_dataset_final['IF_ROTACION_VENTAS'] = pd_dataset_final.apply(lambda row: round((row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] / row['ACTIVO_2023']) if row['ACTIVO_2023'] != 0 else 1, 3), axis=1)

# RENTABILIDAD
pd_dataset_final['IF_MARGEN_BRUTO'] = pd_dataset_final.apply(lambda row: round(((row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] - row['COSTO_VENTAS_PRODUCCION_2023']) / row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023']) if row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] != 0 else 1, 3), axis=1)
pd_dataset_final['IF_MARGEN_OPERACIONAL'] = pd_dataset_final.apply(lambda row: round(((row['UTILIDAD_OPERATIVA_2023']) / row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023']) if row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] != 0 else 1, 3), axis=1)
pd_dataset_final['IF_MARGEN_NETO'] = pd_dataset_final.apply(lambda row: round(((row['UTILIDAD_NETA_2023']) / row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023']) if row['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'] != 0 else 1, 3), axis=1)
pd_dataset_final['IF_ROE'] = pd_dataset_final.apply(lambda row: round((row['UTILIDAD_NETA_2023'] / row['PATRIMONIO_NETO_2023']) if row['PATRIMONIO_NETO_2023'] != 0 else 1, 3), axis=1)
pd_dataset_final['IF_ROA'] = pd_dataset_final.apply(lambda row: round((row['UTILIDAD_NETA_2023'] / row['ACTIVO_2023']) if row['ACTIVO_2023'] != 0 else 1, 3), axis=1)


In [137]:
columns_to_clean_if = ['IF_PRUEBA_ACIDA','IF_ENDEUDAMIENTO_ACTIVO','IF_APALANCAMIENTO','IF_ROTACION_VENTAS','IF_MARGEN_BRUTO','IF_MARGEN_OPERACIONAL','IF_MARGEN_NETO','IF_ROE','IF_ROA']
describe_formateado(pd_dataset_final, columns_to_clean_if)


C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,IF_PRUEBA_ACIDA,IF_ENDEUDAMIENTO_ACTIVO,IF_APALANCAMIENTO,IF_ROTACION_VENTAS,IF_MARGEN_BRUTO,IF_MARGEN_OPERACIONAL,IF_MARGEN_NETO,IF_ROE,IF_ROA
count,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000
mean,77.764,2.449,11.113,9.046,-107.897,-705.795,-25.723,0.174,-0.309
std,6040.373,131.481,708.083,750.081,27839.322,200568.606,4219.651,30.221,16.255
min,-0.192,0.000,-158558.655,0.000,-8203745.625,-59711998.453,-1189623.420,-1970.795,-2619.043
25%,0.697,0.147,1.011,0.273,0.354,0.002,0.000,0.000,0.000
50%,1.075,0.580,1.636,1.083,1.000,0.039,0.000,0.000,0.000
75%,2.772,0.893,4.230,2.584,1.000,0.182,0.055,0.124,0.023
max,1115094.520,23960.594,73572.366,204569.180,1.000,119741.600,117293.210,7647.009,792.476


Aplicar control de outlier Extremos

In [138]:
## Aplicar control de outlier Extremos
pd_dataset_final = eliminar_outliers_tecnica_iqr(pd_dataset_final, columns_to_clean_if, 10)


In [139]:
#Describe luego de limpieza
describe_formateado(pd_dataset_final, columns_to_clean_if)

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,IF_PRUEBA_ACIDA,IF_ENDEUDAMIENTO_ACTIVO,IF_APALANCAMIENTO,IF_ROTACION_VENTAS,IF_MARGEN_BRUTO,IF_MARGEN_OPERACIONAL,IF_MARGEN_NETO,IF_ROE,IF_ROA
count,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000,88673.000
mean,3.712,0.713,4.258,2.655,0.667,0.095,0.074,0.119,0.021
std,6.320,1.091,9.758,4.784,0.606,0.548,0.240,0.377,0.096
min,-0.192,0.000,-31.179,0.000,-6.106,-1.798,-0.550,-1.240,-0.230
25%,0.697,0.147,1.011,0.273,0.354,0.002,0.000,0.000,0.000
50%,1.075,0.580,1.636,1.083,1.000,0.039,0.000,0.000,0.000
75%,2.772,0.893,4.230,2.584,1.000,0.182,0.055,0.124,0.023
max,23.522,8.353,36.420,25.694,1.000,1.982,0.605,1.364,0.253


## Exportar DATASET FINAL COMPLETO para EDA
Se exporta el datset completo, conservando todas las actividades Económicas, para:
+ Realizar un EDA 
+ Calcular el Location Quotient

In [140]:
print(f'Tamaño Dataset: {pd_dataset_final.shape}')
pd_dataset_final.to_csv('Datasets Procesados\\directorio_empresas_final_ciiu_completo.csv', index=False)

Tamaño Dataset: (88673, 71)


## Exportar DATSET FINAL Para CLUSTERING

Se consideran las siguientes categorías CIIU (SECTORES)
+ A - AGRICULTURA, GANADERÍA, SILVICULTURA Y PESCA
+ C - INDUSTRIAS MANUFACTURERAS
+ I - ALOJAMIENTO Y COMIDAS (Servicios)
+ J - INFORMACIÓN Y COMUNICACIÓN. (Servicios)
+ K - ACTIVIDADES FINANCIERAS Y DE SEGUROS. (Servicios)
+ L - ACTIVIDADES INMOBILIARIAS. (Excluida tras primera iteración pruebas)
+ M - Actividades profesionales, científicas y técnicas (Servicios)
+ N - Servicios administrativos y de apoyo (Servicios)

|  | Sección | Descripción Grande                                      | Sector                  |
|----|---------|---------------------------------------------------------|-------------------------|
|✅| A       | Agricultura, ganadería, silvicultura y pesca           | Agricultura, ganadería, silvicultura y pesca |
|   | B       | Explotación de minas y canteras                        | Explotación de minas y canteras |
|✅| C       | Industrias manufactureras                              | Industrias manufactureras |
|   | G       | Comercio al por mayor y al por menor                   | Comercio               |
|   | F       | Construcción                                           | Construcción           |
|   | D       | Suministro de electricidad, gas, vapor                 | Servicios              |
|   | E       | Agua, alcantarillado, desechos y saneamiento           | Servicios              |
|   | H       | Transporte y almacenamiento                            | Servicios              |
|✅| I       | Alojamiento y de servicios de comidas                  | Servicios              |
|✅| J       | Información y comunicación                             | Servicios              |
|✅| K       | Actividades financieras y de seguros                   | Servicios              |
|   | L       | Actividades inmobiliarias                              | Servicios              |
|✅| M       | Actividades profesionales, científicas y técnicas      | Servicios              |
|✅| N       | Servicios administrativos y de apoyo                   | Servicios              |
|   | O       | Administración pública, defensa, seguridad social      | Servicios              |
|   | P       | Enseñanza                                              | Servicios              |
|   | Q       | Atención de la salud humana y asistencia social        | Servicios              |
|   | R       | Artes, entretenimiento y recreación                    | Servicios              |
|   | S       | Otras actividades de servicios                         | Servicios              |



In [141]:
print('Dataset Shape Antes: ', pd_dataset_final.shape)
pd_dataset_final_filtrado = pd_dataset_final[pd_dataset_final['CIIU_NIVEL_1'].isin(['A', 'C', 'I', 'J', 'K', 'M', 'N'])]
print('Dataset Shape Despues: ', pd_dataset_final_filtrado.shape)

Dataset Shape Antes:  (88673, 71)
Dataset Shape Despues:  (37884, 71)


+ Para PCA se eliminan valores extremos en las cuentas de 2022 y 2023, luego de haber generado los Indicadores Financieros y variaciones

In [142]:
columns_revision = ['ACTIVO_2022', 'INVENTARIOS_2022', 'PASIVO_2022','PASIVO_CORRIENTE_2022', 
					'PATRIMONIO_NETO_2022',
                    'INGRESOS_ACTIVIDADES_ORDINARIAS_2022', 'COSTO_VENTAS_PRODUCCION_2022', 'GASTOS_2022',
                    'UTILIDAD_OPERATIVA_2022', 'GANACIA_PERDIDA_ANTES_IR_2022', 'IMPUESTO_RENTA_2022', 'UTILIDAD_NETA_2022',
                    'ACTIVO_2023', 'INVENTARIOS_2023', 'PASIVO_2023', 'PASIVO_CORRIENTE_2023', 
					'PATRIMONIO_NETO_2023',
                    'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'COSTO_VENTAS_PRODUCCION_2023', 'GASTOS_2023',
                    'UTILIDAD_OPERATIVA_2023', 'GANACIA_PERDIDA_ANTES_IR_2023', 'IMPUESTO_RENTA_2023', 'UTILIDAD_NETA_2023']

pd_dataset_final_filtrado = eliminar_outliers_tecnica_iqr(pd_dataset_final_filtrado, columns_revision, 50)

describe_formateado(pd_dataset_final_filtrado, columns_revision)

C:\Users\fabrizio\AppData\Local\Temp\ipykernel_10792\564406641.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_description = description.applymap(lambda x: f"{x:.3f}")


,ACTIVO_2022,INVENTARIOS_2022,PASIVO_2022,PASIVO_CORRIENTE_2022,PATRIMONIO_NETO_2022,INGRESOS_ACTIVIDADES_ORDINARIAS_2022,COSTO_VENTAS_PRODUCCION_2022,GASTOS_2022,UTILIDAD_OPERATIVA_2022,GANACIA_PERDIDA_ANTES_IR_2022,...,PASIVO_2023,PASIVO_CORRIENTE_2023,PATRIMONIO_NETO_2023,INGRESOS_ACTIVIDADES_ORDINARIAS_2023,COSTO_VENTAS_PRODUCCION_2023,GASTOS_2023,UTILIDAD_OPERATIVA_2023,GANACIA_PERDIDA_ANTES_IR_2023,IMPUESTO_RENTA_2023,UTILIDAD_NETA_2023
count,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,...,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000,37884.000
mean,948083.200,0.000,514557.076,296287.096,316810.979,741200.710,206625.980,255293.937,29016.085,18741.564,...,557424.786,339484.443,356135.424,861597.875,282659.498,321450.120,39187.465,22274.424,5987.376,6911.522
std,2935642.976,0.000,1622197.660,938675.465,1009840.590,2350112.493,547648.493,837677.912,131351.831,81975.042,...,1760838.856,1077551.551,1147692.408,2751349.997,788329.453,1044130.646,191879.746,102975.891,17880.943,34509.774
min,0.000,0.000,0.000,0.000,-5242625.880,0.000,0.000,0.000,-647291.625,-369358.125,...,0.000,0.000,-6087484.058,0.000,0.000,0.000,-980863.250,-470851.250,0.000,-119994.500
25%,8037.180,0.000,1564.490,501.127,982.120,3177.250,0.000,270.263,0.000,0.000,...,2752.080,1666.003,1285.067,9299.905,0.000,5766.642,0.000,0.000,0.000,0.000
50%,59892.550,0.000,26955.280,15203.470,13951.575,51867.540,0.000,21792.865,419.440,0.000,...,34248.225,22012.915,18044.155,73562.030,0.000,42105.765,1857.005,39.710,26.065,0.000
75%,355300.422,0.000,201335.480,116935.263,105854.280,300094.445,44496.965,135183.092,12945.833,7387.163,...,224991.280,140964.217,123060.450,372132.660,69777.735,184859.200,19617.265,9417.025,1792.913,2399.890
max,17718462.547,0.000,10189884.980,5938642.013,5349462.280,15145954.195,2269345.215,6880824.592,660237.458,376745.287,...,11336951.280,7105874.968,6211829.575,18513770.410,3558664.485,9139487.075,1000480.515,480268.275,91438.538,122394.390


In [143]:
print(f'Tamaño Dataset: {pd_dataset_final_filtrado.shape}')
pd_dataset_final_filtrado.to_csv('Datasets Procesados\\directorio_empresas_final.csv', index=False)

Tamaño Dataset: (37884, 71)
